<a href="https://colab.research.google.com/github/govardhan-06/SafeScape/blob/main/ML_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video from Youtube for testing

In [ ]:
!pip install yt-dlp

In [10]:
import yt_dlp

def download_youtube(url, output_path='.'):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            print(f"Downloaded: {info['title']}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
short_url = 'https://www.youtube.com/shorts/vOsMflUyums'  # Replace VIDEO_ID with the actual Shorts ID
download_youtube(short_url, output_path='./shorts')  # Downloads to the 'shorts' folder

## Code

In [ ]:
!pip install torch torchvision opencv-python deepface

In [13]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
import cv2
import numpy as np

In [16]:
def person_detection(image):
    model=ssdlite320_mobilenet_v3_large(pretrained=True)
    model.eval()
    transform=T.Compose([
      T.ToTensor()
    ])

    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        predictions = model(image_tensor)
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    # Filter out persons
    person_boxes = [box for i, box in enumerate(boxes) if labels[i] == 1 and scores[i] > 0.5]
    return person_boxes

In [ ]:
from google.colab.patches import cv2_imshow
cap = cv2.VideoCapture("/content/shorts/Bingo! RINGA RINGA ROSA-EP6 ｜ Karikku #shorts.webm")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    person_boxes = person_detection(frame)
    for box in person_boxes:
        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
person_boxes